In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-data-set/Loan_Train.csv


In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
from fastbook import *
from fastai.tabular.all import *

path=Path('/kaggle/input/loan-approval-data-set')
path

Path('/kaggle/input/loan-approval-data-set')

In [4]:
df=pd.read_csv(path/'Loan_Train.csv', low_memory=False)

In [5]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [6]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
df.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [8]:
df['Education'].unique()

array(['Graduate', 'Not Graduate'], dtype=object)

In [9]:
df['Loan_Amount_Term'].unique()

array([360., 120., 240.,  nan, 180.,  60., 300., 480.,  36.,  84.,  12.])

In [10]:
df['Property_Area'].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

Categorify is a TabularProc that replaces a column with a numeric categorical column. FillMissing is a TabularProc that replaces missing values with the median of the column, and creates a new Boolean column that is set to True for any row where the value was missing. These two transforms are needed for nearly every tabular dataset you will use, so this is a good starting point for your data processing:

In [11]:
procs = [Categorify, FillMissing, Normalize]

In [12]:
#df['Gender'] = df['Gender'].astype('category')
#df['Gender'].cat.set_categories(df['Gender'].unique(), ordered=False, inplace=True)

In [13]:
#https://docs.fast.ai/tutorial.tabular.html
dls=TabularDataLoaders.from_csv(path/'Loan_Train.csv', path=path, 
                                y_names='Loan_Status', #label column
                               cat_names = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area'], #category columns
                               cont_names=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History'],#continuous columns
                               procs=procs)

#If you get error message: TypeError: Categorical cannot perform the operation median,
#it is most likely that you have one categorical variable flagged as continuous. 
#So when it tried to find its median value during FillMissing, it throws an error (sgugger,forum.fast.ai, 2018).

In [14]:
#The show_batch method works like for every other application:
dls.show_batch()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount_na,Loan_Amount_Term_na,Credit_History_na,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,Male,No,0,Not Graduate,No,Semiurban,False,False,False,3747.999928,1668.000001,109.999999,360.000001,1.0,Y
1,Male,No,0,Not Graduate,Yes,Semiurban,False,False,False,5799.999978,-0.000069,132.000000,360.000001,1.0,Y
2,Female,No,0,Not Graduate,No,Semiurban,False,False,False,1962.999852,-0.000069,52.999997,360.000001,1.0,Y
3,Male,Yes,2,Graduate,No,Semiurban,False,False,False,6699.999982,1749.999996,230.000000,300.000000,1.0,Y
4,Male,Yes,0,Graduate,No,Rural,False,False,False,7900.999982,1833.000006,180.000001,360.000001,1.0,Y
5,Male,No,0,Graduate,No,Urban,False,False,False,1799.999976,1212.999988,46.999998,360.000001,1.0,Y
6,Male,No,0,Graduate,No,Urban,False,False,False,3749.999918,-0.000069,113.000001,479.999996,1.0,N
7,Male,Yes,1,Graduate,#na#,Semiurban,False,False,True,3716.999980,2924.999976,151.000000,360.000001,1.0,N
8,Male,Yes,2,Graduate,No,Urban,False,False,False,4565.999996,-0.000069,100.000001,360.000001,1.0,N
9,#na#,Yes,0,Graduate,No,Rural,False,False,False,2472.999999,1842.999991,159.000000,360.000001,1.0,N


In [15]:
#splits=RandomSplitter(valid_pct=0.2)

In [16]:
dls.xs.iloc[:2]

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount_na,Loan_Amount_Term_na,Credit_History_na,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
499,2,1,1,1,1,1,1,1,1,0.16898,1.028602,0.867840,0.276644,-2.395774
97,2,2,1,1,1,2,1,1,1,-0.56115,-0.214126,-1.206258,0.276644,0.417402


In [17]:
#We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.
learn=tabular_learner(dls, metrics=accuracy)

In [18]:
#And we can train that model with the fit_one_cycle method (the fine_tune method won't be useful here since we don't have a pretrained model).
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.658113,0.651958,0.827869,00:00


In [19]:
#We can then have a look at some predictions:
learn.show_results()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount_na,Loan_Amount_Term_na,Credit_History_na,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Loan_Status_pred
0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,0.022140,-0.576513,0.333009,0.276644,0.417402,1.0,1.0
1,2.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,0.135068,0.968265,2.446241,0.276644,0.417402,1.0,1.0
2,0.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,-0.288835,-0.576513,-0.606204,0.276644,0.417402,0.0,1.0
3,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,-0.240679,-0.576513,-1.049722,0.276644,0.417402,0.0,1.0
4,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,0.807208,-0.576513,-0.195298,-1.605912,0.417402,1.0,1.0
5,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,0.005015,-0.576513,-0.084419,0.276644,0.417402,1.0,1.0
6,2.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,-0.353776,0.242764,-0.214865,-2.547191,-2.395774,0.0,0.0
7,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.273065,0.986802,0.385188,0.276644,0.417402,1.0,1.0
8,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-0.222367,0.343448,-1.141034,0.276644,0.417402,1.0,1.0


In [20]:
#Or use the predict method on a row:
row, clas, probs = learn.predict(df.iloc[0])

In [21]:
row.show()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount_na,Loan_Amount_Term_na,Credit_History_na,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,Male,No,0,Graduate,No,Urban,True,False,False,5849.00002,-0.000069,127.5,360.000001,1.0,Y


In [22]:
clas, probs

(tensor(1), tensor([0.4643, 0.5357]))

In [23]:
#To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. 
#That dataframe does not need to have the dependent variable in its column.
test_df=df.copy()
test_df.drop(['Loan_Status'], axis=1, inplace=True)

dl=learn.dls.test_dl(test_df)

In [24]:
#Then Learner.get_preds will give you the predictions:
learn.get_preds(dl=dl)

(tensor([[0.4643, 0.5357],
         [0.4603, 0.5397],
         [0.4640, 0.5360],
         ...,
         [0.4621, 0.5379],
         [0.4640, 0.5360],
         [0.5086, 0.4914]]),
 None)

In [25]:
#training more cycles
learn_2 = tabular_learner(dls, metrics=accuracy)
learn_2.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.710477,0.691967,0.393443,00:00
1,0.674749,0.651306,0.827869,00:00
2,0.621014,0.565955,0.819672,00:00
3,0.577503,0.503491,0.827869,00:00
4,0.540219,0.469949,0.836066,00:00
5,0.511462,0.457074,0.844262,00:00
6,0.491505,0.451468,0.836066,00:00
7,0.475469,0.447091,0.836066,00:00
8,0.461398,0.443117,0.827869,00:00
9,0.450455,0.440873,0.827869,00:00


In [26]:
learn_2.show_results()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,LoanAmount_na,Loan_Amount_Term_na,Credit_History_na,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Loan_Status_pred
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.273065,0.986802,0.385188,0.276644,0.417402,1.0,1.0
1,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,-0.057892,-0.576513,-0.195298,0.276644,-2.395774,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.048566,-0.576513,-0.136597,0.276644,0.417402,1.0,1.0
3,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,0.163385,-0.576513,-0.188776,0.276644,0.417402,1.0,1.0
4,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,-0.501295,1.054045,-0.554026,0.276644,0.417402,1.0,1.0
5,2.0,2.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,0.163385,0.039581,0.880884,0.276644,0.417402,1.0,1.0
6,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,-0.517064,-0.576513,-1.036677,2.159201,-2.395774,0.0,0.0
7,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,-0.361407,0.070113,-0.032240,0.276644,0.417402,1.0,1.0
8,2.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.319864,0.332180,-0.253999,0.276644,-2.395774,0.0,0.0
